<a href="https://colab.research.google.com/github/jjups96/mcd_ingenieria_de_caracteristicas/blob/main/Tarea2_Poetas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<p><img src="https://mcd.unison.mx/wp-content/themes/awaken/img/logo_mcd.png" width="150">
</p>



# Curso *Ingeniería de Características*

### Manejo de XML


<p> Jordan Joel Urias Paramo </p>
<p>
<img src="https://identidadbuho.unison.mx/wp-content/uploads/2019/06/letragrama-cmyk-72.jpg" width="150">
</p>


<a target="_blank" href="https://colab.research.google.com/github/mcd-unison/ing-caract/blob/main/ejemplos/integracion/python/descarga_datos.ipynb"><img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Ejecuta en Google Colab</a>

</center>

# 1. primero descargamos la informacion

En la tarea se nos proporciono un link a un repositorio de github que ya tiene la info

In [1]:
#Cargamos librerias base
import os  # Para manejo de archivos y directorios
import urllib.request # Una forma estandard de descargar datos

import datetime # Fecha de descarga
import pandas as pd # Solo para ver el archivo descargado
import zipfile # Descompresión de archivos
import numpy as np

In [2]:
#pw
print(os.getcwd())

poestas_argentinos_url = "https://github.com/mcd-unison/ing-caract/raw/main/ejemplos/integracion/ejemplos/wikipedia-poetas.xml"
poestas_argentinos_archivo = "wikipedia-poetas.xml"
subdir = "./data/"

/content


In [3]:
if not os.path.exists(poestas_argentinos_archivo):
    if not os.path.exists(subdir):
        os.makedirs(subdir)
    urllib.request.urlretrieve(poestas_argentinos_url, subdir + poestas_argentinos_archivo)  


    with open(subdir + "info.txt", 'w') as f:
        f.write("Archivos sobre poetas argentinos\n")
        info = """
        Datos en formato XML que estan feos de leer

        Los datos se obtuvieron del github del profe Waissman con fecha de 03 de Septiembre de 2022

        """ 
        f.write(info + '\n')
        f.write("Descargado el " + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "\n")
        f.write("Desde: " + poestas_argentinos_url + "\n")
        f.write("Nombre: " + poestas_argentinos_archivo + "\n")

# 2. Haciendo trampa

En la tarea dice que descarguemos leyamos el xml de Poetas argentnos de wikipedia. Pero no dice con que libreria.

Asi usare una libreria hecha especificamente hecha para bajar y manejar los archivos xml de wikipedia.

In [4]:
!pip install mwxml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import mwxml
import glob

# 3. Veamos como funciona esta libreria

In [6]:
paths = glob.glob(subdir + poestas_argentinos_archivo)

def process_dump(dump, path):
  for page in dump:
    for revision in page:
        yield page.id,page.title, revision.id, revision.timestamp, len(revision.text)

#Comentado por que ya se que sale, y llena la pantalla
#for page_id,page_title, rev_id, rev_timestamp, rev_textlength in mwxml.map(process_dump, paths):
#    print(";".join(str(v) for v in [page_id, page_title,rev_id, rev_timestamp, rev_textlength]))

# 4. Definimos una problematica y/o que se pueden hacer con los datos
Como aproveche para sacar la informacion usando la libreria de wikimedia para checar los dumps, creo que estaria padre revisar las ediciones a los articulos, por ejemplo checar cuales son los poetas mas editados, cuantas ediciones se hacen por año, en que epocas se hacen mas ediciones. Asi podriamos encontrar algunos patrones interesantes y talvez algunas tendencias.

---

# 5. Creamos un dataframe

Ya que vemos como funciona (la libreria), procedemos a crear un dataframe para organizarlo, y guardarlo en un formato que pueda ser transformado posteriormente a un dataframe, por facilidad guardamos el objeto de tipo timestamp como string con un formato bonito para pandas.

In [7]:
array_data=[]

for page_id,page_title, rev_id, rev_timestamp, rev_textlength in mwxml.map(process_dump, paths):
    array_data.append([page_id, page_title,rev_id, rev_timestamp.strftime("%m/%d/%Y, %H:%M:%S"), rev_textlength])

In [8]:
df = pd.DataFrame(np.array(array_data),
                   columns=['page_id', 'page_title', 'rev_id','rev_timestamp','rev_textlength'])

Checamos los tipos

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639 entries, 0 to 638
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   page_id         639 non-null    object
 1   page_title      639 non-null    object
 2   rev_id          639 non-null    object
 3   rev_timestamp   639 non-null    object
 4   rev_textlength  639 non-null    object
dtypes: object(5)
memory usage: 25.1+ KB


Y convertimos a timestamp en pandas la columna del tiempo

In [10]:
df['rev_timestamp'] = df['rev_timestamp'].apply(pd.Timestamp)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639 entries, 0 to 638
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   page_id         639 non-null    object        
 1   page_title      639 non-null    object        
 2   rev_id          639 non-null    object        
 3   rev_timestamp   639 non-null    datetime64[ns]
 4   rev_textlength  639 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 25.1+ KB


In [13]:
df.head()

,page_id,page_title,rev_id,rev_timestamp,rev_textlength
0,4949229,Julia Morilla de Campbell,102979192,2017-10-30 07:46:11,4081
1,5105749,Luis Negreti,93686970,2016-09-17 17:35:26,5047
2,4477192,Poldy Bird,108610182,2018-06-11 04:34:41,5845
3,423422,Ana María Shua,108670781,2018-06-13 18:50:17,11800
4,4284479,León Benarós,107379800,2018-04-29 21:49:56,9276


# 6. Lo guardamos como parquet.

In [14]:
df.to_parquet('poetas.argentinos.parquet.gzip',
              compression='gzip')

Y pongo el codigo para leer para que no se me olvide

In [15]:
pd.read_parquet('poetas.argentinos.parquet.gzip')

,page_id,page_title,rev_id,rev_timestamp,rev_textlength
0,4949229,Julia Morilla de Campbell,102979192,2017-10-30 07:46:11,4081
1,5105749,Luis Negreti,93686970,2016-09-17 17:35:26,5047
2,4477192,Poldy Bird,108610182,2018-06-11 04:34:41,5845
3,423422,Ana María Shua,108670781,2018-06-13 18:50:17,11800
4,4284479,León Benarós,107379800,2018-04-29 21:49:56,9276
...,...,...,...,...,...
634,7887761,Humberto Tejera,109126317,2018-07-06 17:07:38,24579
635,8440353,Mario Molina Cruz,109468905,2018-07-23 14:27:14,5861
636,6449649,Luis Ignacio Helguera,107708539,2018-05-11 14:19:35,13917
637,8528770,Daniel Olivares Viniegra,109413118,2018-07-20 21:56:20,3205
